In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras import optimizers, metrics
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

root = '../dataset/hi'
root_soh = '../dataset/soh_charge'
root_model = '../model'


In [2]:

def load_and_process(root, root_soh, dataset):
    hiv = pd.read_csv(f'{root}/hiv-charge-CS2_{dataset}.csv')
    soh = pd.read_csv(f'{root_soh}/Csoh-CS2_{dataset}.csv')

    # Mantieni solo le colonne da hiv e soh
    df = pd.merge(hiv, soh, how="right", on="cycle")[["cycle", "hi_v", "SOH"]]
    df = df.drop(columns=["cycle"])
    df_hiv = df[["hi_v"]]
    df_soh = df[["SOH"]]
    return df_hiv.values, df_soh.values

# Carica e processa i dataset
df_33_hiv, df_33_soh = load_and_process(root, root_soh, 33)
df_35_hiv, df_35_soh = load_and_process(root, root_soh, 35)
df_37_hiv, df_37_soh = load_and_process(root, root_soh, 37)
df_36_hiv, df_36_soh = load_and_process(root, root_soh, 36)
df_38_hiv, df_38_soh = load_and_process(root, root_soh, 38)

# Stampa i dataset
print(df_35_hiv.shape)

(797, 1)


In [3]:
seq_length = 10
n_features = 1

def split_sequence(data, seq_length, train=False):

    seq = []
    
    for j in data:
        ##to use all values
        # for i in range(len(j) - seq_length - 1):
        #     if train is True:
        #         seq.append(j[i:i+seq_length])
        #     else:
        #         seq.append(j[i + 1:i+seq_length + 1])
        for i in range(len(j) - seq_length):
            if train is True:
                seq.append(j[i:i+seq_length])
            else:
                seq.append(j[i+seq_length-1])
    if train:
        return np.array(seq)
    else:
        return np.array(seq)

In [ ]:
# Create sequences for training set

X_train = split_sequence([df_36_hiv, df_38_hiv], seq_length, train=True)
y_train = split_sequence([df_36_soh, df_38_soh], seq_length)

# Calculate mean and std for normalization
mean = X_train.mean()
std = X_train.std()

# Standardize train data
X_train = (X_train - mean) / std

# Shuffle the sequences
perms = np.random.permutation(len(X_train))
X_train = X_train[perms]
y_train = y_train[perms]


print(X_train.shape, y_train.shape)

In [13]:
#early stopping
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50)

# Funzione per creare il modello LSTM
def create_lstm_model(units=128, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, activation=activation, input_shape=(seq_length, n_features)))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse', metrics=[metrics.RootMeanSquaredError()])
    return model

keras_model = KerasRegressor(build_fn=create_lstm_model, epochs=1000, verbose=1, callbacks=[es])

# Definisci i parametri da testare
param_grid = {
    'units': [128, 256],
    'activation': ['relu', 'tanh'],
    'optimizer': ['adam'],
    'batch_size': [64, 128]
}

# Esegui la grid search
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/1000


C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_15204\3761182605.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_model = KerasRegressor(build_fn=create_lstm_model, epochs=1000, verbose=1, callbacks=[es])


18/18 [==============================] - 1s 12ms/step - loss: 0.4094 - root_mean_squared_error: 0.6399
Epoch 2/1000
18/18 [==============================] - 0s 11ms/step - loss: 0.0546 - root_mean_squared_error: 0.2337
Epoch 3/1000
18/18 [==============================] - 0s 11ms/step - loss: 0.0070 - root_mean_squared_error: 0.0835
Epoch 4/1000
18/18 [==============================] - 0s 12ms/step - loss: 0.0015 - root_mean_squared_error: 0.0383
Epoch 5/1000
18/18 [==============================] - 0s 15ms/step - loss: 5.7642e-04 - root_mean_squared_error: 0.0240
Epoch 6/1000
18/18 [==============================] - 0s 16ms/step - loss: 3.8099e-04 - root_mean_squared_error: 0.0195
Epoch 7/1000
18/18 [==============================] - 0s 13ms/step - loss: 3.1145e-04 - root_mean_squared_error: 0.0176
Epoch 8/1000
18/18 [==============================] - 0s 13ms/step - loss: 2.6998e-04 - root_mean_squared_error: 0.0164
Epoch 9/1000
18/18 [==============================] - 0s 14ms/step - 

In [17]:
with open("grid_search_results.txt", "w") as f:
    # Write results in file
    f.write("Best parameters:\n")
    f.write(str(grid_result.best_params_) + "\n")
    f.write("Best score:\n")
    f.write(str(grid_result.best_score_) + "\n")
    f.write("\nAll results:\n")
    for params, mean_score, std_score in zip(grid_result.cv_results_['params'], grid_result.cv_results_['mean_test_score'], grid_result.cv_results_['std_test_score']):
        f.write(f"{params}: {mean_score} (+/- {std_score})\n")

In [24]:
activation = grid_result.best_params_["activation"]
batch_size = grid_result.best_params_["batch_size"]
optimizer = grid_result.best_params_["optimizer"]
units = grid_result.best_params_["units"]
epochs = 15000


In [25]:

#define optimizer
adam = optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999)
# define model
model = Sequential()
# to use y_trai as x_train put return_sequences=True
model.add(LSTM(units, activation=activation, input_shape=(seq_length, n_features), return_sequences= False))
model.add(Dense(1))
model.compile(optimizer=optimizer, loss='mse', metrics=[metrics.RootMeanSquaredError()])

c:\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [26]:
# # Create sequences for testing set
X_test = split_sequence([df_35_hiv], seq_length, train = True)
y_test = split_sequence([df_35_soh], seq_length)

# # Normalize the test data using the mean and std from training data
X_test = (X_test - mean) / std

In [27]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=200)

# fit model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data = (X_test, y_test),verbose=1, callbacks=[es])


Epoch 1/15000


14/14 [==============================] - 2s 100ms/step - loss: 0.5484 - root_mean_squared_error: 0.7406 - val_loss: 0.2775 - val_root_mean_squared_error: 0.5268
Epoch 2/15000
14/14 [==============================] - 1s 74ms/step - loss: 0.1230 - root_mean_squared_error: 0.3508 - val_loss: 0.0480 - val_root_mean_squared_error: 0.2191
Epoch 3/15000
14/14 [==============================] - 1s 75ms/step - loss: 0.0386 - root_mean_squared_error: 0.1964 - val_loss: 0.0136 - val_root_mean_squared_error: 0.1168
Epoch 4/15000
14/14 [==============================] - 1s 75ms/step - loss: 0.0098 - root_mean_squared_error: 0.0988 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0402
Epoch 5/15000
14/14 [==============================] - 1s 73ms/step - loss: 0.0012 - root_mean_squared_error: 0.0347 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0331
Epoch 6/15000
14/14 [==============================] - 1s 74ms/step - loss: 7.0195e-04 - root_mean_squared_error: 0.0265 - val_loss: 3.5794e-04

KeyboardInterrupt: 

In [ ]:
model.save(f'{root_model}/LSTM.h5') 
print(history.history.keys())

In [ ]:
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'val')
plt.title('Model Loss - MAE')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.plot(history.history['root_mean_squared_error'], label = 'train')
plt.plot(history.history['val_root_mean_squared_error'], label = 'val')
plt.title('Model Loss - RMSE')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Change test df to plot different samples
file_number = 33
file_hiv = globals()[f'df_{file_number}_hiv']
file_soh = globals()[f'df_{file_number}_soh']
X_test = split_sequence([file_hiv], seq_length, train = True)
y_test = split_sequence([file_soh], seq_length)
X_test = (X_test - mean) / std

eighty_percent_length = int(0.8 * len(X_test))
X_test = X_test[:eighty_percent_length]
y_test = y_test[:eighty_percent_length]

In [ ]:
MSE_train, RMSE_train = model.evaluate(X_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (MSE_train, RMSE_train))
MSE_test, RMSE_test = model.evaluate(X_test, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (MSE_test,RMSE_test))

In [ ]:
# Predici i valori sul dataset di test
y_pred = model.predict(X_test)
mae = np.abs(y_pred - y_test).mean()
rmse = np.sqrt(np.power((y_pred - y_test),2).mean())
print(y_pred.shape, mae*100, rmse*100)
plt.plot((y_test/1.1)*100, label='Real', color='blue')
plt.plot((y_pred/1.1)*100, label='Predicted', color='red')
plt.title(f'Real vs Predicted, Dataset:{file_number}')
plt.xlabel('Cycle')
plt.ylabel('SOH(%)')
plt.legend()
plt.show()